# Libraries

In [2]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange

C:\Users\enhi\AppData\Local\Temp\ipykernel_6556\3827688388.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\FAPAR anomaly\\'
# Directory to the 8days clipped FAPAR Glass to the AOI
FAPAR_Glass = os.path.join(main_dir, 'Clipped_FAPAR')
# Directory to an example of Copernicus FAPAR to take the shape and crs fro it
Copernicus_ex =rxr.open_rasterio(r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\FAPAR anomaly\Monthly_FAPAR_Glass\April\2000April.tif', masked =True)
# Directory to the AOI
Boundary = os.path.join(main_dir, 'Boundary')

In [3]:
# Load the shapefile
def load_shape_file(filepath):
    """Loads the shape file desired to mask a grid.
    Args:
        filepath: Path to *.shp file
    """
    shpfile = gpd.read_file(filepath)
    print("""Shapefile loaded. To prepare for masking, run the function
        `select_shape`.""")
    return shpfile

#Create the mask
def select_shape(shpfile):

    """Select the submask of interest from the shapefile.
    Args:
        shpfile: (*.shp) loaded through `load_shape_file`
        category: (str) header of shape file from which to filter shape.
            (Run print(shpfile) to see options)
        name: (str) name of shape relative to category.
           Returns:
        shapely polygon
    """

    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']

    # Extract the rows that have 'ZAF', 'LSO', or 'SWZ' in the 'SOV_A3' column
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]

    # Combine the selected polygons into a single polygon
    unioned_polygon = selected_rows.geometry.unary_union

    # Convert the unioned polygon to a geopandas dataframe with a single row
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    
    print("""Mask created.""")

    return mask_polygon
shpfile = load_shape_file(Boundary + '\\'+ 'CNTR_RG_01M_2020_4326.shp')
merged = select_shape(shpfile)

Shapefile loaded. To prepare for masking, run the function
        `select_shape`.
Mask created.


In [4]:
# crs of FAPAR copernicus
Copernicus_ex_crs = Copernicus_ex.rio.crs
# Define a crs to the AOI 
merged.crs =Copernicus_ex_crs

In [5]:
'''
The day of the year of the start of the 8 days is extracted from 
the name of FAPAR file. 
There are two functions, one if the start data is in the month and the other
if the end data is in the month.
'''
#Function if the start date of FAPAR is in the month.
def weights_start(Start_date):
    '''
    Inputs:
         date in the format (dd-mm-yyyy)
    returns:
        weight based on the number of observation days overlap the month
    '''
    #Calculation of the number of days on the month 
    num_days = monthrange(int(Start_date.strftime('%Y')),
                          int(Start_date.strftime('%m')))[1]
    # The starting day of the 8days FAPAR in the month
    day_of_month = int(Start_date.strftime('%d'))
    # Number of days bewteen the starting data and the end of the month
    diff = (num_days +1) - day_of_month
    #Ofcourse if the Number of days bewteen the starting data and the end of 
    #the month is greater than 8, that means that all 8 days are in the month
    if diff > 8:
        day_obs = 8 #day_obs: days of observation FAPAR in the month
    #If the difference is less than 8 days that means the days of observation
    #in that month equal to the number of days bewteen the starting data and 
    #the end of the month.
    else: 
        day_obs = diff
    # Calculaion of the weight based on the days of observation FAPAR
    # in the month
    weight = (day_obs / num_days)
    
    return weight
#Function if the end date of FAPAR is in the month.
def weights_end(End_date):
    '''
    Inputs:
         date in the format (dd-mm-yyyy)
    returns:
        weight based on the number of observation days overlap the month
    '''
    #Calculation of the number of days on the month 
    num_days = monthrange(int(End_date.strftime('%Y')),
                          int(End_date.strftime('%m')))[1]
    # The ending day of the 8Days FAPAR in the month
    day_of_month = int(End_date.strftime('%d'))
    # The days of of observation FAPAR is equal to end date,
    # but it will be a further steps
    #in the following lines of code
    day_obs = day_of_month
    
    weight = (day_obs / num_days)
     # Calculaion of the weight based on the days of observation FAPAR
     # in the month
    return weight
    

In [8]:
'''
The code is built based on that there are years folders which contain all
the 8days FAPAR data in that year
'''
months = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'Aug', 'Sep',
          'Oct', 'Nov', 'Dec']
#list of all years of 8days historical data
years = sorted(os.listdir(FAPAR_Glass))
#Empty list to contain other lists of the monthly FAPAR data for
#each month through years
#for example, list of monthly FAPAR of January of each year
months_through_years = []
#looping through 12 months, the looping started by months not years 
#as the goal is to calculate histrocal max and min values for each month
count = 0
for month in range(1,13):

    #Empty list for the monthly FAOAR data for each month through years
    lis_of_m =[]
    #looping through years
    for year in years:
        #Empty list to contain all 8days FAPAR overlapping in the month
        month_in_year = []
        #looping through the 8days FAPAR of the year
        for image in os.listdir(FAPAR_Glass + '\\' + year):
            #Read the year from name of the file
            y = image[16:20]
            #Read the day of the year from the name of the file
            DOY = image[20:23]
            #convert the day of the year to a date in the format (DD-MM-yyyy)
            DOY.rjust(3 + len(DOY), '0')
            Date_of_start = datetime.strptime(y + "-" + DOY, "%Y-%j")
            Date_of_end = Date_of_start + timedelta(days=8)
            #If the starting date is in the month
            if int(Date_of_start.strftime('%m')) == month:
                #Apply the function to calculate the weight based on 
                #the starting date
                weight = weights_start(Date_of_start)
                
                #The path of the image
                image = FAPAR_Glass + '\\' + year + '\\' + image
                #Open the tiff file of 8days FAPAR
                FAPAR = rxr.open_rasterio(image,
                                  masked=True)
                # getting scale factor
                Scale_factor = FAPAR.attrs['scale_factor']
                # Calculate real FAPAR values using scale factor
                FAPAR =FAPAR * Scale_factor
                #Multiply the 8days FAPAR with the weight 
                img_weighted = weight * FAPAR
                #Append th weighted 8days FAPAR to the list of all images 
                #in that month
                month_in_year.append(img_weighted)
            #If the ending date is in the month  
            elif int(Date_of_end.strftime('%m')) == month:
                # If the end day is after 8th of the month,
                #that means the image was already considered in starting date,
                #so nothing will happen.
                if int(Date_of_end.strftime('%d')) >7:
                    continue
                #If the end day is in January, it should not be considered as
                #the first image of a year is always starts in 1st of January 
                if int(Date_of_end.strftime('%m')) == 1:
                    continue
                    
                #Apply the function to calculate the weight based on
                #the ending date
                weight = weights_end(Date_of_end)
                #The path of the image
                image = FAPAR_Glass + '\\'+ year + '\\' + image
                #Open the tiff file of 8days FAPAR
                FAPAR = rxr.open_rasterio(image,
                                  masked=True)
                # getting scale factor
                Scale_factor = FAPAR.attrs['scale_factor']
                # Calculate real FAPAR values using scale factor
                FAPAR =FAPAR * Scale_factor

                 #Multiply the 8days FAPAR with the weight
                img_weighted = weight * FAPAR
                
                #Append th weighted 8days FAPAR to the list of all images
                #in that month
                month_in_year.append(img_weighted)
              
        #Calculating the monthly composite by adding
        #the weighted averaged 8days FAPAR overlap this month            
        monthly_composite = sum(month_in_year)
        
        '''
        the following condition is to deal with years which has months with no 8days FAPAR data
        like 2000 which the data starts by the end of February. At this time the monthly_composite
        will be integer, so the following line is to skip these values
        '''
        if type(monthly_composite) != int:
            # Reprojection and resampling of FAPAR Glass to match Copernicus FAPAR
            # The data resamlpled from 250m to 300m using nearest neighbour
            monthly_composite= monthly_composite.rio.reproject(Copernicus_ex.rio.crs,
                                        shape = Copernicus_ex.rio.shape,
                                        resampling=Resampling.nearest).rio.reproject_match(Copernicus_ex)
            
            # Export monthly FAPAR Glass
            monthly_composite.rio.to_raster(main_dir + '\\' + 'Monthly_FAPAR_Glass'+'\\' +year + months[count] + '.tif')

    count = count + 1


C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\